# FCNN Baseline Classifier for Plant Waves
----

#### PREPROCESSING:

In [1]:
import numpy as np
import torch
import pickle
import os
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

%run ../src/utils/constants.py

Load raw data that has been split into train, test, and validation sets for the 81k and 191k cases.

In [2]:
path_to_pickle_train81k = DATASETS_DIR / "train-81k-stratified-without-neutral.pkl"
path_to_pickle_train191k = DATASETS_DIR / "train-191k-stratified-without-neutral.pkl"
path_to_pickle_test81k = DATASETS_DIR / "test-81k-stratified-without-neutral.pkl"
path_to_pickle_val81k = DATASETS_DIR / "val-81k-stratified-without-neutral.pkl"

In [ ]:
with open(path_to_pickle_train81k, 'rb') as train_81k_file:
    train_81k = pickle.load(train_81k_file)

In [3]:
with open(path_to_pickle_train191k, 'rb') as train_191k_file:
    train_191k = pickle.load(train_191k_file)

In [4]:
with open(path_to_pickle_test81k, 'rb') as test_81k_file:
    test_81k = pickle.load(test_81k_file)

In [5]:
with open(path_to_pickle_val81k, 'rb') as val_81k_file:
    val_81k = pickle.load(val_81k_file)

Just in case you want to use the 191k dataset, run the following line of code. Don't load the `train_81k_file`.

In [6]:
train_81k = train_191k
len(train_81k)

28742

While ignoring the class `Neutral`, separate wave slices and labels, and create a Tensor dataset for train, test, val.

##### Train 81k

In [7]:
wav_slices_train_81k = []
labels_train_81k = []
for segment_train_81k in train_81k:  
    wav_slices_train_81k.append(segment_train_81k[0])
    labels_train_81k.append(segment_train_81k[1])

In [8]:
wav_slices_train_81k = torch.tensor(np.array(wav_slices_train_81k), dtype=torch.float32)
labels_train_81k = torch.tensor(np.array(labels_train_81k), dtype=torch.long)
dataset_train_81k = TensorDataset(wav_slices_train_81k, labels_train_81k)

Normalize the train samples using per-sample normalization.

In [9]:
data_tensor_train_81k, labels_tensor_train_81k = dataset_train_81k.tensors
mean_train_81k = torch.mean(data_tensor_train_81k, dim=1, keepdim=True)
std_dev_train_81k = torch.std(data_tensor_train_81k, dim=1, keepdim=True)
standardized_data_train_81k = (data_tensor_train_81k - mean_train_81k) / (std_dev_train_81k + 0.00000001)

dataset_train_81k = TensorDataset(standardized_data_train_81k, labels_tensor_train_81k)

##### Test 81k

In [10]:
wav_slices_test_81k = []
labels_test_81k = []
for segment_test_81k in test_81k:  
    wav_slices_test_81k.append(segment_test_81k[0])
    labels_test_81k.append(segment_test_81k[1])

In [11]:
wav_slices_test_81k = torch.tensor(np.array(wav_slices_test_81k), dtype=torch.float32)
labels_test_81k = torch.tensor(np.array(labels_test_81k), dtype=torch.long)
dataset_test_81k = TensorDataset(wav_slices_test_81k, labels_test_81k)

Normalize the test samples using per-sample normalization.

In [12]:
data_tensor_test_81k, labels_tensor_test_81k = dataset_test_81k.tensors
mean_test_81k = torch.mean(data_tensor_test_81k, dim=1, keepdim=True)
std_dev_test_81k = torch.std(data_tensor_test_81k, dim=1, keepdim=True)
standardized_data_test_81k = (data_tensor_test_81k - mean_test_81k) / (std_dev_test_81k + 0.00000001)

dataset_test_81k = TensorDataset(standardized_data_test_81k, labels_tensor_test_81k)

##### Validation 81k

In [13]:
wav_slices_val_81k = []
labels_val_81k = []
for segment_val_81k in val_81k:  
    wav_slices_val_81k.append(segment_val_81k[0])
    labels_val_81k.append(segment_val_81k[1])

In [14]:
wav_slices_val_81k = torch.tensor(np.array(wav_slices_val_81k), dtype=torch.float32)
labels_val_81k = torch.tensor(np.array(labels_val_81k), dtype=torch.long)
dataset_val_81k = TensorDataset(wav_slices_val_81k, labels_val_81k)

Normalize the validation samples using per-sample normalization.

In [15]:
data_tensor_val_81k, labels_tensor_val_81k = dataset_val_81k.tensors
mean_val_81k = torch.mean(data_tensor_val_81k, dim=1, keepdim=True)
std_dev_val_81k = torch.std(data_tensor_val_81k, dim=1, keepdim=True)
standardized_data_val_81k = (data_tensor_val_81k - mean_val_81k) / (std_dev_val_81k + 0.00000001)

dataset_val_81k = TensorDataset(standardized_data_val_81k, labels_tensor_val_81k)

#### Create dataloaders

In [16]:
batch_size = 32

In [17]:
train81k_loader = DataLoader(dataset_train_81k, batch_size)
test81k_loader = DataLoader(dataset_test_81k, batch_size)
val81k_loader = DataLoader(dataset_val_81k, batch_size)

#### Hyperparameter Optimization

In [18]:
import wandb
import math
import optuna
from optuna.trial import TrialState
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score

C:\Users\nekta\OneDrive\Desktop\botanic-emotion-recognition\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define the FCNN model.

In [19]:
def define_model(trial):
    layers = []

    INPUT_DIM = 10000
    N_CLASSES = 6
    dropout_rate = trial.suggest_categorical("dropout_rate", [0.0, 0.1, 0.2])

    hidden_dim_1 = trial.suggest_categorical("hidden_dim_1", [2 ** i for i in range(4, 8)])

    layers.append(torch.nn.Linear(INPUT_DIM, hidden_dim_1))
    layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Dropout(dropout_rate))

    hidden_dim_2 = trial.suggest_categorical("hidden_dim_2", [2 ** i for i in range(4, 7)])
    
    layers.append(torch.nn.Linear(hidden_dim_1, hidden_dim_2))
    layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Dropout(dropout_rate))
    
    layers.append(torch.nn.Linear(hidden_dim_2, N_CLASSES))

    config_dict = {
        "hidden_dim_1": hidden_dim_1,
        "hidden_dim_2": hidden_dim_2,
        "dropout_rate": dropout_rate
    }

    return torch.nn.Sequential(*layers), config_dict

Define the objective function for optuna for the hyperparameter optimization.

In [20]:
def objective(trial): 
    model, config_dict = define_model(trial)

    lr = trial.suggest_categorical('lr', [0.0001, 0.001, 0.01])

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    print(config_dict)
    name_experiment = (f"{trial.number}_{NAME_CORE}_lr-{lr}_hidden1-{config_dict['hidden_dim_1']}_hidden2-"
                       f"{config_dict['hidden_dim_2']}_dr-{config_dict['dropout_rate']}")

    config_dict["lr"] = lr
    config_dict["epochs"] = NUM_EPOCHS

    wandb.init(
        project="model_" + NAME_CORE + "-hpo",
        dir=LOGS_DIR,
        name=name_experiment,
        config=config_dict
    )


    for epoch in range(NUM_EPOCHS):
        model.train()
        for batch_data, batch_labels in tqdm(train81k_loader):          
            # zero gradients
            optimizer.zero_grad()

            # forward + backward pass
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        all_preds = []
        all_labels = []

        model.eval()
        with torch.no_grad():
            for batch_data, batch_labels in tqdm(val81k_loader): 
                output = model(batch_data)

                predicted = output.argmax(dim=1)
                all_preds.extend(predicted.numpy())
                all_labels.extend(batch_labels.numpy())

        # TODO: explicitely state in written thesis that zero_division=0.0
        balanced_accuracy = balanced_accuracy_score(all_labels, all_preds)
        accuracy = accuracy_score(all_labels, all_preds)
        f1_class = f1_score(all_labels, all_preds, average=None, zero_division=0.0)
        f1_weighted = f1_score(all_labels, all_preds, average="weighted", zero_division=0.0)
        recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0.0)
        precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0.0)

        trial.report(balanced_accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        metrics = {
            "balanced_accuracy": balanced_accuracy,
            "accuracy": accuracy,
            "f1_weighted": f1_weighted,
            "recall": recall,
            "precision": precision
        }

        wandb_input = metrics
        wandb.log(wandb_input)

    wandb.finish()

    return balanced_accuracy

Run hyperparameter optimization using optuna. Specify a study name and define the search space for the grid search. 

In [21]:
search_space = {
    'lr': [0.0001, 0.001, 0.01],
    'hidden_dim_1': [2 ** i for i in range(4, 8)],
    'hidden_dim_2': [2 ** i for i in range(4, 7)],
    "dropout_rate": [0, 0.1, 0.2]
}

In [22]:
num_combi = 1
for i in search_space:
    num_combi *= len(search_space[i])
num_combi

108

In [23]:
NAME_CORE = f"fcnn_baseline_6_large_81k_search_{num_combi}"
NUM_EPOCHS = 20

#### Second Run:

In [24]:
sampler = optuna.samplers.GridSampler(search_space)
study = optuna.create_study(sampler=sampler, study_name=NAME_CORE, storage="sqlite:///hpo_" + NAME_CORE + ".db",
                            direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=num_combi)

[I 2024-02-14 20:49:20,989] A new study created in RDB with name: fcnn_baseline_6_large_81k_search_108


{'hidden_dim_1': 16, 'hidden_dim_2': 64, 'dropout_rate': 0}


wandb: Currently logged in as: totikos. Use `wandb login --relogin` to force relogin


100%|█████████████████████████████████████████| 82/82 [00:00<00:00, 702.89it/s]


accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
balanced_accuracy,▁███████████████████
f1_weighted,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.23846
balanced_accuracy,0.16667
f1_weighted,0.09183
precision,0.05686
recall,0.23846


...

[I 2024-02-14 22:18:01,509] Trial 105 finished with value: 0.16763639665089147 and parameters: {'dropout_rate': 0.1, 'hidden_dim_1': 32, 'hidden_dim_2': 64, 'lr': 0.0001}. Best is trial 6 with value: 0.17126425545780385.


{'hidden_dim_1': 32, 'hidden_dim_2': 16, 'dropout_rate': 0.1}


100%|█████████████████████████████████████████| 82/82 [00:00<00:00, 745.28it/s]
[I 2024-02-14 22:18:07,683] Trial 106 pruned. 


{'hidden_dim_1': 64, 'hidden_dim_2': 64, 'dropout_rate': 0.1}


100%|█████████████████████████████████████████| 82/82 [00:00<00:00, 490.61it/s]
[I 2024-02-14 22:18:21,259] Trial 107 pruned. 


In [25]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [26]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("  Number of pruned trials: ", len(pruned_trials))

print("Best trial:")
trial_ = study.best_trial

print("  Value: ", trial_.value)
print("  Params: ")
for key, value in trial_.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  108
  Number of complete trials:  26
  Number of pruned trials:  82
Best trial:
  Value:  0.17126425545780385
  Params: 
    dropout_rate: 0.2
    hidden_dim_1: 64
    hidden_dim_2: 16
    lr: 0.001


#### 191k RUN 
Study statistics: 
  Number of finished trials:  108
  Number of complete trials:  26
  Number of pruned trials:  82
Best trial:
  Value:  0.17126425545780385
  Params: 
    dropout_rate: 0.2
    hidden_dim_1: 64
    hidden_dim_2: 16
    lr: 0.001

#### First Run:

In [ ]:
sampler = optuna.samplers.GridSampler(search_space)
study = optuna.create_study(sampler=sampler, study_name=NAME_CORE, storage="sqlite:///hpo_" + NAME_CORE + ".db",
                            direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=num_combi)

Closer look into the trials. 

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("  Number of pruned trials: ", len(pruned_trials))

print("Best trial:")
trial_ = study.best_trial

print("  Value: ", trial_.value)
print("  Params: ")
for key, value in trial_.params.items():
    print("    {}: {}".format(key, value))

#### Training the Baseline FCNN.
10 times training with the best HP from HPO, and then average. 

#### Testing the Baseline Classifier

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

%run ../src/utils/constants.py

In [ ]:
test_81k_folder = "test-81k"